In [ ]:
import pandas as pd

from data import *

# Filler value for logically skipped ones
filler_value = -999

# Define the conditions dictionary for each column of the education module
# Format: { column_name: { 'value': (skip_to_column_index, action_type) } }
education_conditions = {
    "q03": {2: "skip_row", "NO": "skip_row"},
    "Q08": {2: "Q11", "NO": "Q11"},
    "Q09": {1: "Q12", "YES": "Q12"},
    "Q10": {range(1, 14): "Q14"},
    "Q11": {range(1, 15): "Q14"},
    "Q14": {2: "Q17", "NO": "Q17"},
    "Q15": {1: "Q21", "YES": "Q21"},
    "Q16": {range(1, 14): "Q20"},
    "Q19": {2: "skip_row", "NO": "skip_row"},
    "Q20": {range(1, 3): "skip_row", "NO": "skip_row"},
    "Q24": {lambda x: x < 5: "Q26", "ABROAD": "Q33", 999: "Q33"},
    "Q28": {range(1, 4): "Q32", "WALK": "Q32", "BICYCLE": "Q32", "ANIMAL": "Q32"},
    "Q30": {2: "Q32", "NO": "Q32"},
    "Q43": {1: "Q45", "YES": "Q45"},
    "Q46": {2: "Q50", "NO": "Q50"},
    "Q48": {4: "Q50", "STILL HAVE NOT RECEIVED THE SUBSIDY": "Q50"},
    "Q50": {2: "Q57", "NO": "Q57"},
    "Q54": {2: "Q57", "NO": "Q57"},
    "Q57": {2: "Q59", "NO": "Q59"},
    "Q59": {2: "Q61", "NO": "Q61"},
    "Q61": {2: "Q64", "NO": "Q64"},
    "Q64": {2: "Q66", "NO": "Q66"},
}

house_conditions = {
    "q05y": {range(12): "q09"},
    "q06": {range(4, 6): "q09", "WIDOW/ER": "q09", "SINGLE": "q09"},
    "q07": {2: "q09", "NO": "q09"},
    "q11": {2: "q13", "NO": "q13"},
    "q12": {"not_null": "q17"},
    "q14": {1: "q16", "YES": "q16"},
    "q15": {"not_null": "q17"},
    "q17": {2: "q19", "NO": "q19"},
    "q18": {"not_null": "skip_row"},
    "q20": {1: "q22", "YES": "q22"},
    "q21": {"not_null": "skip_row"},
}


# Function to apply conditions based on the dictionary with column names on the education module.
# Define new category (filler_value), for those which should be skipped.
def apply_conditions(df, conditions):
    def apply_action(df, idx, col_name, action):
        """Helper function to apply the specified action."""
        start_col_index = df.columns.get_loc(col_name)
        if action == "skip_row":
            # Set all columns to the right (from col_name to end of row) to filler_value
            df.loc[idx, df.columns[start_col_index + 1 :]] = filler_value
            return True  # Indicate that the row should be skipped
        elif isinstance(action, str) and action in df.columns:
            # Skip to a specific column within the row and set cells to 'skipped'
            end_col_index = df.columns.get_loc(action)
            df.loc[idx, df.columns[start_col_index + 1 : end_col_index]] = filler_value
            return False

    for idx, row in df.iterrows():
        for col_name, condition in conditions.items():
            if col_name in df.columns:
                value = row[col_name]

                # Check if there’s a condition for the value in this column
                for cond_value, action in condition.items():
                    if isinstance(cond_value, range):
                        if value in cond_value:
                            if apply_action(df, idx, col_name, action):
                                break
                        elif value == filler_value or pd.isna(value):
                            continue
                    elif callable(cond_value):
                        if cond_value(value):
                            if apply_action(df, idx, col_name, action):
                                break
                    elif (value == cond_value) or (
                        cond_value == "not_null" and pd.notna(value)
                    ):
                        if apply_action(df, idx, col_name, action):
                            break
    return df


# Apply the conditions to the DataFrame
df_edu = apply_conditions(edu_train, education_conditions)
df_house = apply_conditions(house_train, house_conditions)
df_edu_test = apply_conditions(edu_test, education_conditions)
df_house_test = apply_conditions(house_test, house_conditions)

In [2]:
import os


df_edu.to_csv(os.path.join(PROCESSED_DIR, "education_train.csv"), index=False)
df_house.to_csv(os.path.join(PROCESSED_DIR, "household_train.csv"), index=False)
df_edu_test.to_csv(os.path.join(PROCESSED_DIR, "education_test.csv"), index=False)
df_house_test.to_csv(os.path.join(PROCESSED_DIR, "household_test.csv"), index=False)

In [3]:
df_house["psu_hh_idcode"] = (
    df_house["psu"].astype(str)
    + "_"
    + df_house["hh"].astype(str)
    + "_"
    + df_house["idcode"].astype(str)
)
df_edu["psu_hh_idcode"] = (
    df_edu["psu"].astype(str)
    + "_"
    + df_edu["hh"].astype(str)
    + "_"
    + df_edu["idcode"].astype(str)
)

df_house_prefixed = df_house.drop(columns=["psu", "hh", "idcode", "hhid"]).add_prefix(
    HOUSE_PREFIX
)
df_edu_prefixed = df_edu.drop(columns=["psu", "hh", "idcode"]).add_prefix(EDU_PREFIX)

# remove prefix on the key merge column "psh_hh_idcode"
df_house_prefixed = df_house_prefixed.rename(
    columns={"house_psu_hh_idcode": "psu_hh_idcode"}
)
df_edu_prefixed = df_edu_prefixed.rename(columns={"edu_psu_hh_idcode": "psu_hh_idcode"})

combined_transformed_train = pd.merge(
    pd.merge(
        pov_train,
        df_house_prefixed,
        on="psu_hh_idcode",
        how="left",
        suffixes=[None, "_house"],
    ),
    df_edu_prefixed,
    on="psu_hh_idcode",
    how="left",
    suffixes=[None, "_edu"],
)

combined_transformed_train.columns = combined_transformed_train.columns.str.lower()
assert combined_transformed_train.shape[0] == pov_train.shape[0]  # same number of rows

# save to csv
combined_transformed_train.to_csv(
    os.path.join(PROCESSED_DIR, "combined_transformed_train.csv"), index=False
)

In [ ]:
# now test data
df_house_test["psu_hh_idcode"] = (
    df_house_test["psu"].astype(str)
    + "_"
    + df_house_test["hh"].astype(str)
    + "_"
    + df_house_test["idcode"].astype(str)
)
df_edu_test["psu_hh_idcode"] = (
    df_edu_test["psu"].astype(str)
    + "_"
    + df_edu_test["hh"].astype(str)
    + "_"
    + df_edu_test["idcode"].astype(str)
)
df_house_test_prefixed = df_house_test.drop(
    columns=["psu", "hh", "idcode", "hhid"]
).add_prefix("house_")
df_edu_test_prefixed = df_edu_test.drop(columns=["psu", "hh", "idcode"]).add_prefix(
    "edu_"
)

# remove prefix on the key merge column "psh_hh_idcode"
df_house_test_prefixed = df_house_test_prefixed.rename(
    columns={"house_psu_hh_idcode": "psu_hh_idcode"}
)
df_edu_test_prefixed = df_edu_test_prefixed.rename(
    columns={"edu_psu_hh_idcode": "psu_hh_idcode"}
)

combined_test = pd.merge(
    pd.merge(
        sample_submission,
        df_house_test_prefixed,
        on="psu_hh_idcode",
        how="left",
        suffixes=[None, "_house"],
    ),
    df_edu_test_prefixed,
    on="psu_hh_idcode",
    how="left",
    suffixes=[None, "_edu"],
)
combined_test.columns = combined_test.columns.str.lower()
assert combined_test.shape[0] == sample_submission.shape[0]  # same number of rows

# save to csv
combined_test.to_csv(
    os.path.join(PROCESSED_DIR, "combined_transformed_test.csv"), index=False
)
combined_test.shape

(1334, 99)